# 01. Языковые модели. Предсказание следующего слова

Одной из задач обработки текстов на естественном языке выступает задача предсказания
следующего слова по предшествующим. Примерами таких задач являются задача
предсказания следующего слова при наборе текста в смартфоне или дополнение поисковых
запросов.

## 01.1. Предсказание по предыдущему слову

Данная задача может быть сведена к оценке *вероятностей встретить* каждое из возможных
слов после имеющегося. Соответствующая языковая модель примет вид:

$$\large w^{*} = \underset{w_k \in V}{\operatorname{argmax}} \mathbb{P}\left[w_k | w_{k-1}\right] \qquad (1)$$

где условная вероятность высчитывается по формуле Байеса:

$$\large \mathbb{P}\left[w_k | w_{k-1}\right]= \frac{\mathbb{P}\left[w_k\right] \cdot \mathbb{P}\left[w_{k-1} | w_k\right]}{\mathbb{P}\left[w_{k-1}\right]} \qquad (2)$$

где

- $V$ - множество всех возможных слов
- $\mathbb{P}\left[w_{k} | w_{k-1}\right]$ - вероятность встретить слово $w_k$ после $w_{k-1}$

Для удобства полученную модель можно логарифмировать и представить в следующем виде:

$$\large w^{*} = \underset{w_k \in V}{\operatorname{argmax}} \left[\log \mathbb{P} \left[w_k \right] + \mathbb{P}\left[w_{k-1} | w_{k}\right]\right] \qquad (3)$$

Условные вероятности в формуле $(3)$ могут быть оценен непосредственно по имеющемуся словарю слов. Для этого необходимо рассмотреть имеющиеся в словаре *биграммы*, тогда:

$$\large \mathbb{P}\left[w_{k-1} | w_{k}\right] = \frac{\nu_{(w_{k-1}, w_k)}}{\sum\limits_{(w_i, w_j) \in V} \nu_{(w_i, w_j)}} \qquad (4)$$

где $\nu_{(w_{k-1}, w_k)}$ - частота встречаемости словосочетания $(w_{k-1}, w_k)$

## 01.2. Предсказание по $m$ последних слов

Иногда по последнему слову оказывается невозможным определить следующее, поскольку в нём не учитывается контекст. Например, если последнее слово является союзом, то приемлемые варианты следующего слова определяет и предшествующее данному союзу слово. Оценка вероятности в данном случае проводится на основе $m$ последних слов. 

Таким образом, модель $(1) - (2)$ принимает вид:

$$\large w^{*} = \underset{w_k \in V}{\operatorname{argmax}} \mathbb{P}\left[w_k | w_{k-1}, w_{k-2}, \ldots, w_{k-m}\right] \qquad (5)$$

где условная вероятность высчитывается по формуле Байеса:

$$\large \mathbb{P}\left[w_k | w_{k-1}, w_{k-2}, \ldots, w_{k-m}\right]= \frac{\mathbb{P}\left[w_k\right] \cdot \mathbb{P}\left[w_{k-1}, w_{k-2}, \ldots, w_{k-m} | w_k\right]}{\mathbb{P}\left[w_{k-1}, w_{k-2}, \ldots, w_{k-m}\right]} \qquad (6)$$

### 01.2.1 Независимость от порядка слов

Предположим, что текущее слово $w_k$ зависит только то того, какие слова встретились перед ним и не зависит от того, в каком порядке они встретились. Тогда:

$$\large w^{*} = \underset{w_k \in V}{\operatorname{argmax}} \mathbb{P}\left[w_k\right] \cdot \prod\limits_{i=1}^m \mathbb{P}\left[w_{k-i} | w_k \right] = \underset{w_k \in V}{\operatorname{argmax}} \left[\log \mathbb{P}\left[w_k\right] + \sum\limits_{i=1}^m \log \mathbb{P}\left[w_{k-i} | w_k \right]\right]\qquad (7)$$

где $\mathbb{P}\left[w_{k-i} | w_k \right]$ вычисляются по формуле $(4)$

### 01.2.2. Учёт порядка предшествующих слов

Языковая модель $(7)$ не учитывает порядок предшествующих слов. Информацию о порядке слов можно добавить путём оценки расстояния до каждого из предшествующих слов. Например, в предложении "Счастье есть удовольствие без раскаяния" расстояние между словами "счастье" и "раскаяния" равно $4$.

In [1]:
import numpy as np
import pandas as pd
import nltk
import collections
import re
import string
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
def ngrams(sentense: list, n: int):
    '''
    sentense: list 
    n: ngram
    
    Выводит сначала 1-gram, потом 2-gram и.т.д - удобно
    
    >>> test = ['cat','sat','on']
    >>> ngrams(test, 3)
    <<< ['cat', 'sat', 'on', 'cat sat', 'sat on', 'cat sat on']
    '''
    
    result = []
    fullgram = len(sentense)
    for i in range(fullgram):
        if i == n:
            break
        for j in range(fullgram-i):
            result.append(' '.join(sentense[j:i+1+j]).rstrip('.'))
    return result

def text_prepare(text):
    """
        text: a string

        return: modified string
    """
    # Перевести символы в нижний регистр
    text = text.lower()

    # Заменить символы пунктуации на пробелы
    text = re.sub(r'[{}]'.format(string.punctuation), '', text)

    # Удалить "плохие" символы
    text = re.sub('[^A-Za-z0-9 ]', '', text)
    return text

In [3]:
text = '''
Backgammon is one of the oldest known board games. 
Its history can be traced back nearly 5,000 years to archeological discoveries in the Middle East. 
It is a two player game where each player has fifteen checkers 
which move between twenty-four points according to the roll of two dice.
Backgammon do board games every, night fight it in. 
'''

In [ ]:
file_name = 
with open(file_name, 'r', encoding='utf-8') as file:
    text = file.readlines()

In [3]:
def read_data(file_name='war_peace_processed.txt'):
    data = open(file_name, 'rt', encoding='utf-8').read()
    return data.split('\n')

In [4]:
text = read_data()

In [5]:
text

['1',
 'в',
 'два',
 'раза',
 'короче',
 'и',
 'в',
 'пять',
 'раз',
 'интереснее',
 '2',
 'почти',
 'нет',
 'философических',
 'отступлений',
 '3',
 'в',
 'сто',
 'раз',
 'легче',
 'читать',
 'весь',
 'французский',
 'текст',
 'заменен',
 'русским',
 'в',
 'переводе',
 'самого',
 'толстого',
 '4',
 'гораздо',
 'больше',
 'мира',
 'и',
 'меньше',
 'войны',
 '5',
 'хеппи-энд',
 'эти',
 'слова',
 'я',
 'поместил',
 'семь',
 'лет',
 'назад',
 'на',
 'обложку',
 'предыдущего',
 'издания',
 'указав',
 'в',
 'аннотации',
 'первая',
 'полная',
 'редакция',
 'великого',
 'романа',
 'созданная',
 'к',
 'концу',
 '1866',
 'года',
 'до',
 'того',
 'как',
 'толстой',
 'переделал',
 'его',
 'в',
 '1867--1869',
 'годах',
 '--',
 'и',
 'что',
 'я',
 'использовал',
 'такие-то',
 'публикации',
 'думая',
 'что',
 'все',
 'всё',
 'знают',
 'я',
 'не',
 'объяснил',
 'откуда',
 'взялась',
 'эта',
 'первая',
 'редакция',
 'я',
 'оказался',
 'неправ',
 'и',
 'в',
 'результате',
 'оголтелые',
 'и',
 'невежест

In [23]:
text.index('уважение')

1164

In [32]:
text = ''.join(text[1167:])

In [ ]:
n = 4
words_sentences = [sentence.lower().split() for sentence in nltk.sent_tokenize(text)]
result = [ngrams(snt, n) for snt in words_sentences]
flatten = lambda l: [item for sublist in l for item in sublist]
words = flatten(result)
prepared_words = [text_prepare(word) for word in words]
count = collections.Counter(prepared_words)

In [9]:
count

Counter({'1': 16,
         '': 299163,
         '2': 18,
         '3': 20,
         '4': 15,
         '5': 8,
         '1866': 3,
         '18671869': 1,
         '1863': 1,
         '726': 2,
         '1805': 22,
         '18681869': 1,
         '1870': 1,
         '1983': 1,
         '94': 1,
         '30': 6,
         '1873': 1,
         'new': 170,
         'chapter': 170,
         '7': 8,
         '10': 11,
         'a': 170,
         '40': 3,
         '000': 7,
         'xvi': 2,
         'he': 1,
         '18': 3,
         'moscou': 2,
         '1796': 2,
         'po': 1,
         'm': 2,
         'chpre': 2,
         'mn': 1,
         'cher': 1,
         'maman': 3,
         '1809': 13,
         'hy': 2,
         '700': 1,
         'o': 9,
         'un': 2,
         'btard': 1,
         'ax': 6,
         'aa': 1,
         'p': 1,
         'abc': 1,
         'ps': 1,
         '13': 4,
         '12': 14,
         'xo': 1,
         'xaxaxa': 1,
         '8': 2,
         '3000': 4